# Gemini API を使用して、生成AIをPythonで利用する

### 環境変数の設定（Google Colabの場合）

1. Google ColabのシークレットタブからGoogle APIキーの「Google AI Studioでキーを管理」という項目をクリックしてGemini APIキーの管理画面に移動します
2. 管理画面から「APIキーの作成」を行い、APIキーを追加します
3. Google Colabに戻り、シークレットタブからGoogle APIキーの「Google AI Studioからキーをインポート」をクリックして先ほど追加したAPIキーを選択します
4. シークレットタブに「GOOGLE_API_KEY」という名前が追加されているのを確認したら以下のセルを実行し、エラーが発生しないことを確認してください

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None

### 必要なライブラリをインストールします


In [ ]:
!pip install google-genai requests

### requestsライブラリを使用したGemini APIの呼び出し

In [ ]:
import requests
import json

# 使用するモデルのIDを指定します
model_id = "gemini-1.5-flash"
# APIエンドポイントのURLを指定します
API_ENDPOINT = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent"

prompt = "こんにちは！"

headers = {
    "Content-Type": "application/json",
}
"""
payload (辞書)
└── "contents" (リスト)
    └── [0]番目の要素 (辞書)  ← 会話の1ターン目（ユーザー入力）
        └── "parts" (リスト)
            └── [0]番目の要素 (辞書) ← コンテンツの1要素（テキスト）
                └── "text" (文字列) ← 実際のプロンプトテキスト (例: "こんにちは！")
"""

payload = {"contents": [{"parts": [{"text": prompt}]}]}
params = {"key": GOOGLE_API_KEY}

try:
    response = requests.post(
        API_ENDPOINT, headers=headers, params=params, data=json.dumps(payload)
    )
    response.raise_for_status()  # HTTPエラーがあれば例外を発生させる

    response_data = response.json()
    # 応答の内容を確認する
    print(response_data)

except requests.exceptions.HTTPError as http_err:
    print(f"HTTPエラー: {http_err}")

except Exception as err:
    print(f"その他のエラー: {err}")

{'candidates': [{'content': {'parts': [{'text': 'こんにちは！\n\n何かお手伝いできることはありますか？\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.03462708989779154}], 'usageMetadata': {'promptTokenCount': 2, 'candidatesTokenCount': 12, 'totalTokenCount': 14, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 2}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 12}]}, 'modelVersion': 'gemini-1.5-pro-002'}


### 【演習1】Gemini APIのレスポンスからAIの応答メッセージを抽出する
生のデータから応答メッセージだけを抽出し、出力しましょう

In [ ]:
BASE_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/"

def get_gemini_response(prompt, model_id="gemini-1.5-flash"):
    """
    Google Gemini APIにリクエストを送信し、モデルからの応答テキストを取得します。

    Args:
        prompt (str): モデルに送信するプロンプト（指示文）。
        model_id (str, optional): 使用するモデルのID。デフォルトは "gemini-1.5-flash"。

    Returns:
        dict[str, any]: モデルからの応答を含む辞書。成功時は応答データ
    """
    api_url = f"{BASE_API_URL}{model_id}:generateContent"
    headers = {
        "Content-Type": "application/json",
    }
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    params = {"key": GOOGLE_API_KEY}
    

    response = requests.post(api_url, headers=headers, params=params, data=json.dumps(payload))

    try:
        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    except Exception as e:
        print(f"APIリクエスト中にエラーが発生しました: {e}")
        raise e

prompt = "こんにちは！自己紹介をしてくれますか？"
response = get_gemini_response(prompt)

In [ ]:
# 応答からテキストを抽出して表示


### Google Gen AI SDKを使用したGemini APIの呼び出し

In [5]:
from google import genai
prompt = "PythonでHTTPリクエストを送信する方法を教えてください。"

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model=model_id, contents=prompt
)
print(response.text)

PythonでHTTPリクエストを送信する方法はいくつかありますが、主な方法は以下のとおりです。

**1. `urllib.request` (Python 3 標準ライブラリ):**

シンプルなリクエストに適しています。

```python
import urllib.request

url = "https://www.example.com"

try:
    response = urllib.request.urlopen(url)
    html = response.read().decode('utf-8')  # レスポンスボディを取得
    print(html)

except urllib.error.URLError as e:
    print(f"Error: {e.reason}")
```

**`urllib.request` を使ったPOSTリクエスト:**

```python
import urllib.request
import urllib.parse

url = "https://www.example.com/submit"
data = {'key1': 'value1', 'key2': 'value2'}
encoded_data = urllib.parse.urlencode(data).encode('utf-8')

req = urllib.request.Request(url, data=encoded_data)  # data引数にPOSTデータを渡す
req.add_header('Content-Type', 'application/x-www-form-urlencoded')  # Content-Typeヘッダーを追加

try:
    response = urllib.request.urlopen(req)
    html = response.read().decode('utf-8')
    print(html)

except urllib.error.URLError as e:
    print(f"Error: {e.reason}")
```


**2. `requests` (外部ライブラリ):**

より高度な機能が必要な場合

### 温度（Temperature）の設定
「温度」は、AIの創造性を調整するパラメータです。値が低いと（0に近いと）予測可能で安定した回答になり、値が高いと（1に近いと）より創造的でバラエティに富んだ回答になります。

In [ ]:
from google import genai
from google.genai import types

prompt = "面白い冗談を1つ教えてください"

client = genai.Client(
    api_key=GOOGLE_API_KEY,
    model=model_id,
    prompt=prompt,
    config=types.GenerationConfig(
        temperature=0.5,  # 出力の多様性を制御する温度パラメータ
    )
)

print(response.text)

### 最大出力トークン数
AIの回答の長さを制限したい場合は、「max_output_tokens」を設定します。

In [ ]:
from google import genai
from google.genai import types

prompt = "Pythonの再帰関数について教えてください。"

client = genai.Client(
    api_key=GOOGLE_API_KEY,
    model=model_id,
    prompt=prompt,
    config=types.GenerationConfig(
        max_output_tokens=100,  # 出力の最大トークン数
    ),
)

print(response.text)


### システム指示（System Instruction）
AIに特定の役割や振る舞いを指示したい場合は、「system_instruction」を使います。

In [ ]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="過去の思い出話をしてください",
    config=types.GenerateContentConfig(
        system_instruction="あなたは関西弁を話すおばあちゃんです",
        temperature=0.7,  # 出力の多様性を制御する温度パラメータ
        max_output_tokens=150,  # 出力の最大トークン数
    ),
)

print(response.text)


### チャットボットの作成

単発の質問だけでなく、会話を続けるチャットボットを作ることもできます。これには「チャットセッション」を使います。

### ストリーミング生成

In [6]:
for chunk in client.models.generate_content_stream(
    model=model_id,
    contents=prompt
):
    print(chunk.text, end="", flush=True)

PythonでHTTPリクエストを送信する方法はいくつかあります。最も一般的な方法は、`requests`ライブラリを使用することです。`requests`は使いやすく、様々な機能を提供しています。他に組み込みの`urllib`ライブラリも使用できますが、`requests`の方が簡潔で直感的に使用できるため、一般的に推奨されます。

**1. `requests`ライブラリを使う方法 (推奨)**

```python
import requests

# GETリクエスト
response = requests.get("https://www.example.com")

# ステータスコードを確認
if response.status_code == 200:
    # レスポンスボディ (テキスト) を取得
    print(response.text)
    # レスポンスボディ (バイト列) を取得
    # print(response.content)
    # レスポンスヘッダーを取得
    print(response.headers)
else:
    print(f"リクエストが失敗しました: {response.status_code}")


# POSTリクエスト (データを送信)
data = {'key1': 'value1', 'key2': 'value2'}
response = requests.post("https://www.example.com/submit", data=data)

# JSONデータを送信
import json
data = {'key1': 'value1', 'key2': 'value2'}
response = requests.post("https://www.example.com/submit", json=data)  # Content-Type: application/json が自動的に設定される

# カスタムヘッダーを送信
headers = {'User-Agent': 'My User Agent'}
response = requests.get("https://www.example.com", headers=headers)

# 

### ツールの使用

In [10]:
from google.genai import types

def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "雨"


response = client.models.generate_content(
    model=model_id,
    contents="東京都の天気を教えて",
    config=types.GenerateContentConfig(
        tools=[get_current_weather],
    )
)

print(response.text)

東京は雨です。
